To show this demo, first run

code/ColorMNIST/00_make_data.py

to create the biased MNIST dataset

In [ ]:

import sys
import numpy as np

import pickle as pkl
import torch
from os.path import join as oj
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import torch.utils.data as utils
import sys, time
import pickle as pkl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('precision', 3)

In [ ]:
save_path = "../results_for_export"
x_train = np.load(oj("../data/ColorMNIST", "train_x.npy"))/255.0
y_train = np.load(oj("../data/ColorMNIST", "train_y.npy"))
x_test = np.load(oj("../data/ColorMNIST", "test_x.npy"))/255.0
y_test = np.load(oj("../data/ColorMNIST", "test_y.npy"))

In [ ]:
fig, axes = plt.subplots(nrows= 2, figsize = (20, 4))
indices = []
for i in range(10):
    indices.append(np.where(y_train ==i)[0][0])
ex_imgs = np.hstack(x_train[np.asarray(indices)].swapaxes(1,3).swapaxes(1,2)) 
axes[0].imshow(ex_imgs);
axes[0].set_ylabel("Training", fontsize = 25)
indices = []
for i in range(10):
    indices.append(np.where(y_test ==i)[0][0])
ex_imgs = np.hstack(x_test[np.asarray(indices)].swapaxes(1,3).swapaxes(1,2)) 
axes[1].imshow(ex_imgs);
axes[1].set_xticks([])
axes[1].set_yticks([])
axes[0].set_xticks([])
axes[0].set_yticks([])
axes[1].set_ylabel("Test", fontsize = 25)
# fig.savefig(oj(save_path, "ColorMNIST_examples"))


Now we train a neural network with and without CDEP on this data. To do this, run

code/ColorMNIST/02_make_demo.py 

with GPU enabled (The code was only tested on GPU and may not work on CPU).

This will train without any regularization and with CDEP lambda of 5000.

In [ ]:
trained_model_folder = '../models/ColorMNIST_test' #the models are saved in this folder
fnames = [oj(trained_model_folder, fname) for fname in os.listdir(trained_model_folder)]
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects() 

In [ ]:
sys.path.append('./ColorMNIST')
from model import Net
def test( model, test_loader,):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(0), target.to(0)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred).long()).sum().item()

    return correct/len(test_loader.dataset)*100
model = Net();
model.cuda();

In [ ]:
x_test -= x_test.mean(axis= (0,2,3) )[None, :, None, None,] #normally this happens with the mean and std of the training set, i.e. a fixed value
x_test /= x_test.std(axis= (0,2,3) )[None, :, None, None,]

val_x_tensor = torch.Tensor(x_test)
val_y_tensor = torch.Tensor(y_test)
val_dataset = utils.TensorDataset(val_x_tensor,val_y_tensor)
val_loader = utils.DataLoader(val_dataset,
        batch_size=1000, shuffle=True) 

In [ ]:
acc_list = []
for i in range(len(results)):
    model.load_state_dict(results['model_weights'][i])
    acc_list.append(test(model, val_loader))
results['Test accuracy'] = [x for x in acc_list]

In [ ]:
results[['regularizer_rate','Test accuracy',]]

Regularization with CDEP increases the accuracy on the (inversely biased with color) test set.